In [1]:
import pandas as pd
import pandasql as ps

In [2]:
demand = pd.read_csv('demand.csv')
charge = pd.read_csv('charge.csv')

<ipython-input-2-c8b0d8325621>:1: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  demand = pd.read_csv('demand_test.csv')


In [3]:
demand.DEMAND_ID.nunique() == demand.shape[0]

False

In [4]:
query1 = '''
select count(*)
from
    (
    select * from demand
    where LOAN_ISSUED = 1) d
    left join charge c on d.DEMAND_ID = c.DEMAND_ID
    where CHARGE_SUMMA < 0

'''

In [5]:
# десять платежей с отрицательной суммой
clients_with_negative_CHARGE_SUMMA = ps.sqldf(query1)
clients_with_negative_CHARGE_SUMMA

,count(*)
0,10


In [6]:
query2 = '''
select d.DEMAND_ID, SAVE_STAMP, CHARGE_DATE, LOAN_AMOUNT, CHARGE_SUMMA
from
    (
    select * from demand
    where LOAN_ISSUED = 1) d
    left join charge c on d.DEMAND_ID = c.DEMAND_ID
    where CHARGE_DATE < SAVE_STAMP

'''

In [7]:
# 1392 случая, когда заявка сохранена после первого платежа и иногда даже не одного
clients_with_negative_CHARGE_SUMMA = ps.sqldf(query2)
clients_with_negative_CHARGE_SUMMA

,DEMAND_ID,SAVE_STAMP,CHARGE_DATE,LOAN_AMOUNT,CHARGE_SUMMA
0,56493,2019-05-07 00:00:00,2019-05-06 00:00:00,13000,390.0
1,56558,2019-05-07 00:00:00,2019-05-06 00:00:00,10000,10150.0
2,56609,2019-05-07 00:00:00,2019-05-06 00:00:00,10900,1900.0
3,24376,2019-05-01 00:00:00,2019-04-30 00:00:00,9000,395.0
4,24376,2019-05-01 00:00:00,2019-04-30 00:00:00,9000,9135.0
...,...,...,...,...,...
1387,655890,2019-08-26 00:00:00,2019-08-25 00:00:00,15000,150.0
1388,655890,2019-08-26 00:00:00,2019-08-25 00:00:00,15000,150.0
1389,655890,2019-08-26 00:00:00,2019-08-25 00:00:00,15000,1050.0
1390,655890,2019-08-26 00:00:00,2019-08-25 00:00:00,15000,15000.0


#### 2 варианта

1. Исключить платежи до сохранения заявки:

In [8]:
query3 = '''
with base as 
    (
    select
        d.DEMAND_ID id,
        date(SAVE_STAMP, 'start of month') loan_date,
        LOAN_AMOUNT loan_amount,
        date(CHARGE_DATE, 'start of month') charge_date,
        CHARGE_SUMMA charge_amount,
        round((julianday(date(CHARGE_DATE, 'start of month')) - julianday(date(SAVE_STAMP, 'start of month'))) / 30) period
    from
        (
        select * from demand
        where LOAN_ISSUED = 1
        ) d
    left join charge c
    on d.DEMAND_ID = c.DEMAND_ID
    where CHARGE_SUMMA > 0 and CHARGE_DATE >= SAVE_STAMP -- отфильтруем "некорректные" данные
    )    
        

select
    cp.coghort,
    cp.period,
    borrowers,
    sum(payments) over (partition by cp.coghort order by cp.period
                rows between unbounded preceding and current row) / loans_amount * 100 payments_percentage
    
from
(
select
    loan_date coghort,
    period,
    count(id) borrowers,
    sum(charge_amount) payments
from base
group by
    coghort,
    period
) cp
left join
(
select
    loan_date coghort,
    sum(loan_amount) loans_amount
from (select distinct id, loan_date, loan_amount from base)
group by coghort
) ls

on ls.coghort = cp.coghort
'''

In [9]:
cohorts_1 = ps.sqldf(query3)

cohorts_1.head(60)

,coghort,period,borrowers,payments_percentage
0,2019-05-01,0.0,18863,38.024277
1,2019-05-01,1.0,24353,95.485641
2,2019-05-01,2.0,11042,117.836328
3,2019-05-01,3.0,9244,134.208233
4,2019-05-01,4.0,5106,144.688504
5,2019-05-01,5.0,2694,150.439482
6,2019-05-01,6.0,1543,153.869488
7,2019-05-01,7.0,782,155.329746
8,2019-05-01,8.0,455,156.074738
9,2019-05-01,9.0,423,157.031445


In [10]:
cohorts_1_var = cohorts_1.pivot(index='coghort',
                            columns='period',
                            values=['payments_percentage'])
cohorts_1_var

payments_percentage                                                 \
period                    0.0        1.0         2.0         3.0         4.0    
coghort                                                                         
2019-05-01           38.024277  95.485641  117.836328  134.208233  144.688504   
2019-06-01           39.442670  99.253897  122.491137  136.226906  145.163959   
2019-07-01           36.613054  99.242784  121.511692  132.359483  140.399332   
2019-08-01           33.123746  94.626160  117.670158  129.091165  139.462510   

                                                                        ...  \
period            5.0         6.0         7.0         8.0         9.0   ...   
coghort                                                                 ...   
2019-05-01  150.439482  153.869488  155.329746  156.074738  157.031445  ...   
2019-06-01  150.141552  153.126662  154.487836  155.621308  155.932229  ...   
2019-07-01  145.210477  147.096346  148.489887  148.898800  149.137932  ...   
2019-08-01  143.130095  146.564697  147.583527  147.908166  148.036066  ...   

                                                                        \
period            28.0        29.0        30.0        31.0        32.0   
coghort                                                                  
2019-05-01  170.579249  171.367356         NaN  172.060476  172.825215   
2019-06-01  168.652764  169.305327  169.978027         NaN  170.411872   
2019-07-01  160.425422  161.030919         NaN  161.333886  161.998385   
2019-08-01  161.479272  161.955479         NaN  162.777219  163.217822   

                                                                        
period            33.0        34.0        35.0        36.0        37.0  
coghort                                                                 
2019-05-01  173.226888  173.627422  174.142887  174.485276  174.496343  
2019-06-01  171.229958         NaN  171.615661  171.637774         NaN  
2019-07-01         NaN  162.290539  162.332911         NaN         NaN  
2019-08-01  163.243059         NaN         NaN         NaN         NaN  

[4 rows x 38 columns]

2. Отнять пару дней от даты сохранения заявки, на случай если данные корректны, и эти записи должны принадлежать к апрельским когортам. 

In [11]:
query4 = '''
with base as 
    ( 
    select
        id,
        loan_date,
        loan_amount,
        charge_date,
        charge_amount,
        round((julianday(charge_date) - julianday(loan_date)) / 30) period
    from (
            select
                d.DEMAND_ID id,
                case when CHARGE_DATE < SAVE_STAMP then date(SAVE_STAMP, '-2 days', 'start of month')
                else date(SAVE_STAMP, 'start of month')
                end as loan_date, -- перенесем даты сохранения заявки в предыдущий месяц.
                LOAN_AMOUNT loan_amount,
                date(CHARGE_DATE, 'start of month') charge_date,
                CHARGE_SUMMA charge_amount
            from
                (
                select * from demand
                where LOAN_ISSUED = 1
                ) d
            left join charge c
            on d.DEMAND_ID = c.DEMAND_ID
            where CHARGE_SUMMA > 0
    )    
        )

select
    cp.coghort,
    cp.period,
    borrowers,
    sum(payments) over (partition by cp.coghort order by cp.period
                rows between unbounded preceding and current row) / loans_amount * 100 payments_percentage
    
from
(
select
    loan_date coghort,
    period,
    count(id) borrowers,
    sum(charge_amount) payments
from base
group by
    coghort,
    period
) cp
left join
(
select
    loan_date coghort,
    sum(loan_amount) loans_amount
from (select distinct id, loan_date, loan_amount from base)
group by coghort
) ls

on ls.coghort = cp.coghort
'''

In [12]:
cohorts_2 = ps.sqldf(query4)

In [13]:
cohorts_2_var = cohorts_2.pivot(index='coghort',
                            columns='period',
                            values=['payments_percentage'])
cohorts_2_var

payments_percentage                                                 \
period                    0.0        1.0         2.0         3.0         4.0    
coghort                                                                         
2019-04-01           23.654551  36.559594         NaN         NaN         NaN   
2019-05-01           38.283984  95.636561  117.944934  134.285843  144.746273   
2019-06-01           39.721017  99.461889  122.671314  136.390641  145.316997   
2019-07-01           36.989102  99.478345  121.694944  132.517255  140.538220   
2019-08-01           33.617761  95.034628  118.046574  129.451695  139.808614   

                                                                        ...  \
period            5.0         6.0         7.0         8.0         9.0   ...   
coghort                                                                 ...   
2019-04-01         NaN         NaN         NaN         NaN         NaN  ...   
2019-05-01  150.486363  153.909875  155.367369  156.110951  157.065846  ...   
2019-06-01  150.288631  153.270168  154.629713  155.761828  156.072376  ...   
2019-07-01  145.338064  147.219502  148.609770  149.017723  149.256293  ...   
2019-08-01  143.471097  146.900922  147.918335  148.242522  148.370245  ...   

                                                                        \
period            28.0        29.0        30.0        31.0        32.0   
coghort                                                                  
2019-04-01         NaN         NaN         NaN         NaN         NaN   
2019-05-01  170.588001  171.374617         NaN  172.066424  172.829715   
2019-06-01  168.777685  169.429467  170.101362         NaN  170.534687   
2019-07-01  160.517270  161.121344         NaN  161.423600  162.086538   
2019-08-01  161.794751  162.270296         NaN  163.090893  163.530883   

                                                                       
period            33.0        34.0        35.0        36.0       37.0  
coghort                                                                
2019-04-01         NaN         NaN         NaN         NaN        NaN  
2019-05-01  173.230628  173.630403  174.144893  174.486633  174.49768  
2019-06-01  171.351794         NaN  171.737035  171.759122        NaN  
2019-07-01         NaN  162.378006  162.420278         NaN        NaN  
2019-08-01  163.556086         NaN         NaN         NaN        NaN  

[5 rows x 38 columns]